# Lecture 4: Applications Markov processes

In [ ]:

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl

In [ ]:
def get_filename(filename: str, lecture_id: int = 1, file_extension: str = '.png') -> str:
    return f"L{lecture_id}_{filename}{file_extension}"

outdir = '../figures/'
lecture_id = 4

In [ ]:
'''
------------------------------------------
            SETTINGS
------------------------------------------
'''
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-v0_8-white')
plt.rcParams['font.family'] = 'PT Sans'
# plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 14
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 12

dpi = 100

In [ ]:
def plot_trajectories(trajectory: np.ndarray, x_max:int, x_min: int=0):
    """
    Plots the trajectory of all particles over time.

    Parameters:
    all_positions (ndarray): Positions of particles over all time steps.
    x_min (int): Lower boundary of the 1D line.
    x_max (int): Upper boundary of the 1D line.
    num_particles (int): Number of particles.
    """

    # Plot the trajectory of all particles
    plt.figure(figsize=(10, 6))
    plt.plot(trajectory, '-.', marker='o', alpha=0.5,lw=1)  # Plot trajectory of each particle

    # Add boundary lines
    plt.axhline(y=x_min, color='r', linestyle='--', label=f'Boundary at {x_min}',lw=1)
    plt.axhline(y=x_max, color='g', linestyle='--', label=f'Boundary at {x_max}',lw=1)
    
    plt.title(f'Boundaries ({x_min} to {x_max}) starting at 0\np={p:.3f}, q={q:.3f}')
    plt.xlabel('Step')
    plt.ylabel('Position')
    plt.legend()
    plt.show()

# General random walk

For a general random walk with $q_{i+1}=q$ and $p_i=p$ for all $i\geq 0$, we have:

$
\pi_{i}= \frac{(1-\rho)\,\rho^{i}}{1-\rho^{N+1}}
$

where $\rho=p/q$.

Let's look at how this function changes with $i$, to see how the steady state probability of each state varies

In [ ]:
def calculate_pi_i(i: int, rho: float = 1, N: int = 100):
    if np.isclose(rho,1):
        return 1/(N+1)
    num = (1-rho) * np.power(rho,i)
    den = 1 - np.power(rho,N+1)
    return num / den

In [ ]:
p = 0.505
q = 0.485
r = 1 - (p+q)
rho = p/q
print(f"rho={rho:.4f}, r={r:.2f}")

In [ ]:
N = 10 # number of states - 1
xs = np.arange(N+1)
ys0 = [calculate_pi_i(i,rho=rho,N=N) for i in xs]
assert np.allclose(np.sum(ys0),1)

In [ ]:
fs = 20
plt.figure(figsize=(6,3))

plt.scatter(xs,ys0)
plt.text( N-2,(max(ys0)- 0.1*(max(ys0)-min(ys0))),f"{r'$\rho=$'}{rho:.4f}")
plt.xlabel('State i',fontsize=fs)
plt.ylabel(r'$\pi_i$',fontsize=fs)
plt.show()

How does it change with $\rho$?

### Describe the process investigating the transition probability

We know that the transition probability is:

$
P =\left[ 
\begin{array}{cccccccc} 
r_{0} & p_{0} & 0 &0 &\dots & 0 & 0& 0\\
q_{1} & r_{1} & p_{1} &0 &\dots & 0 & 0& 0\\
0 & q_{2} & r_{2} &p_{2} &\dots & 0 & 0& 0\\
\vdots&\vdots & \vdots &\vdots&\vdots & \vdots & \vdots& \vdots\\
0 & 0& 0 &0 &\dots & q_{N-1} & r_{N-1}& p_{N-1}\\
0 & 0& 0 &0 &\dots & 0&q_{N} & r_{N}
\end{array}
\right] \quad,
$

### Get P

In [ ]:
def get_P(p: float, q: float, N: int = 10):
    '''
    Assuming constant p,q,r
    '''
    r = 1 - (p+q)
    P = np.zeros((N+1,N+1))

    # Boundaries
    P[0,0] = r
    P[0,1] = p
    P[N,N-1] = q
    P[N,N] = r

    # All other entries
    for i in range(1,N):
        P[i,i-1] = q
        P[i,i] = r
        P[i,i+1] = p
    return P

In [ ]:
N = 10
# p = 0.9
# q = 0.1
P = get_P(p, q, N=N)

In [ ]:
fs = 10
plt.figure(figsize=(8,5))
plt.imshow(P, vmax=1,vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(P):
    plt.text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
    plt.text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
plt.axis('off')
plt.colorbar()

### Check that it is a valid probability
This is always the first step to take, as a sanity check.

We ask that the row-sum of $P$ equals 1:
- $\sum_j P_{ij} =1$, $\forall i$

In [ ]:
assert np.all(np.isclose(np.sum(P,axis=1),1)),np.sum(P,axis=1)

Boundaries cannot have the same $p,q$ as the non-boundaries!   

Let's correct this.

In [ ]:
def get_P(p: float, q: float, N: int = 10):
    '''
    Assuming constant p,q,r
    '''
    r = 1 - (p+q)
    P = np.zeros((N+1,N+1))

    # Boundaries
    P[0,0] = r
    P[0,1] = 1-r
    P[N,N-1] = 1-r
    P[N,N] = r

    P[0,0] = 1-p
    P[0,1] = p
    P[N,N-1] = q
    P[N,N] = 1-q

    # All other entries
    for i in range(1,N):
        P[i,i-1] = q
        P[i,i] = r
        P[i,i+1] = p
    return P

In [ ]:
N = 10
# p = 0.8
# q = 0.1
P = get_P(p, q, N=N)

fs = 10
plt.figure(figsize=(8,5))
plt.imshow(P, vmax=1,vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(P):
    plt.text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
    plt.text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
plt.axis('off')
plt.colorbar()

assert np.all(np.isclose(np.sum(P,axis=1),1)),np.sum(P,axis=1)

### Visualize into a Markov graph
Now that we have $P$, we can visualize using a **graph** (or network).  
This will be handy to use all the methods and tools available from network modeling.   
In particular, we will use the [`networkx`](https://networkx.org/) python module.

In [ ]:
filename = 'markov_chain_grw'
filename = get_filename(filename,lecture_id=lecture_id)

outfile = filename
outfile

Let's first define a function to takes in input the matrix $P$ and builds a `networkx` graph object `G` with: 
- **nodes**: states of the Markov chain  
- **edge** weights: entries of $P$

In [ ]:
def get_graph_from_transition(P: np.ndarray,states: list=None) -> nx.MultiDiGraph():
    G = nx.MultiDiGraph()
    if states is None:
        states = np.arange(P.shape[0])
    assert P.shape[0] == len(states) 
    for start_idx, node_start in enumerate(states):
        for end_idx, node_end in enumerate(states):
            value = P[start_idx][end_idx]
            if value != 0:
                G.add_edge(node_start,node_end, weight=value)
    return G

Let's visualize this graph.

In [ ]:
G = get_graph_from_transition(P)


pos = nx.spring_layout(G, seed=10)
fig, ax = plt.subplots()
nx.draw_networkx_nodes(G, pos, node_size=1000, edgecolors='black', node_color='white')
nx.draw_networkx_labels(G, pos, font_size=12)

arc_rad = 0.2

edges = nx.draw_networkx_edges(G, pos, ax=ax, connectionstyle=f'arc3, rad = {arc_rad}', edge_cmap=cm.Blues, width=5,
    edge_color=[G[nodes[0]][nodes[1]][0]['weight'] for nodes in G.edges])

pc = mpl.collections.PatchCollection(edges, cmap=cm.Blues)

ax = plt.gca()
ax.set_axis_off()
plt.colorbar(pc, ax=ax)
plt.tight_layout()
# plt.show()
if outfile is not None:
    
    plt.savefig(f"{outdir}{outfile}", dpi=dpi, format=None, metadata=None,
                bbox_inches='tight', pad_inches=0.1,
                facecolor='auto', edgecolor='auto',
                backend=None
                )
    print(f"Figure saved in {outdir}{outfile}")
        
plt.show()

### Irreducible vs reducible

Q: is this irreducible?

We can use the network property of a graph being **strongly connected** to check this.

In [ ]:
print(f"Is irreducible?\n{nx.is_strongly_connected(G)}!")

## Simulate a Markov Chain
We are now ready to play with the Markov chain and simulate one.  
We can use two methods:
1. Concatenate the transition matrix $P$ as done in Notebook 2
2. Sample binary steps $Z_i \in {-1,0,1}$ with probability $q,r,p$ and add them up

In [ ]:
# Method 1
def get_trajectory_with_P(initial_state: int, transition_matrix: np.ndarray, T: int, seed: int = 10):
    prng = np.random.RandomState(seed=seed)
    # assert np.all(np.isclose(np.sum(transition_matrix,axis=1),1))
    N = transition_matrix.shape[0]
    assert 0 <= initial_state < N
    states = np.zeros(T).astype(int)
    current_state = initial_state
    i = 0
    reached_absorbing = False
    while (i < T) and reached_absorbing == False:
        states[i] = prng.choice(np.arange(N), p=transition_matrix[current_state])
        current_state = states[i]
        
        if np.isclose(transition_matrix[current_state][current_state],1):
            reached_absorbing = True
            states[i+1:] = states[i]
        else:
            i += 1
            
    return states

In [ ]:
# Method 2
def random_walk_reflecting(p: float, q: float, num_steps:int = 100,
                           x_min: int = 0, x_max: int =10,x0: int = None, 
                           boundary_type='reflect',
                          seed: int = 10):
    """
    Simulates a 1D random walk with reflecting or absorbing boundaries

    Parameters:
    num_steps (int): Total number of steps for the random walk.
    x_min (int): Lower boundary of the 1D line.
    x_max (int): Upper boundary of the 1D line.
    boundary_type (str): Type of boundary behavior: 'stick' for stick at the boundary,
                         'reflect' for reflecting at the boundary.

    Returns:
    trajectory (ndarray): states over all time steps.
    """
    assert 0<=p<=1
    assert 0<=q<=1
    r = 1 - (p+q)

    prng = np.random.RandomState(seed=seed)
    
    if x0 is None: x0 = x_min
    # Initialize particles at position 0 (start from zero)
    position = x0
    trajectory = np.zeros(num_steps + 1)  # Store positions for all particles over time
    trajectory[0] = position  # Initial position
    for t in range(1, num_steps + 1):
        step = prng.choice([-1,0, 1], size=1, p=[q,r,p])  # Random step
        position += step  # Update position

        # Apply boundary conditions based on 'sticking_type'
        if boundary_type == 'stick':
            # Apply stick boundaries: process stops at the boundary
            position = np.where(position < x_min, x_min, position)
            position = np.where(position > x_max, x_max, position)

        elif boundary_type == 'reflect':
            # Apply reflecting boundaries: absorbing bounces back from the boundary
            position = np.where(position < x_min, x_min + (x_min - position), position)
            position = np.where(position > x_max, x_max - (position - x_max), position)

        # Store position at each time step
        trajectory[t] = position

    return trajectory

Now we can use this to run it for various time steps.

In [ ]:
states = np.arange(N+1)
initial_state = 0  # Starting state
print(f"Starting state = {states[initial_state]}")
T = 10000 # number of timesteps to simulate

In [ ]:
trajectory_P = get_trajectory_with_P(initial_state,P, T)

In [ ]:
fig, ax = plt.subplots(1,min(T,11),figsize=(6 * min(T,11), 5))

for i in range(11):

    if i > 0:
        old_state = trajectory_P[i-1]
    else:
        old_state = initial_state
    current_state = trajectory_P[i]
    node_color = ['white' for i in range(len(states))]
    node_color[old_state] = 'r'    
    
    nx.draw_networkx_nodes(G, pos, node_size=3000, edgecolors='black', node_color=node_color,ax=ax[i],alpha=0.8)
    nx.draw_networkx_labels(G, pos, font_size=12, ax=ax[i])

    edge_color = ['black' for e in G.edges()]
    width = [1 for e in G.edges]
    for idx, (u,v) in enumerate(G.edges()):
        if (u == states[old_state]) & (v == states[current_state]):
            edge_color[idx] = 'r'
            width[idx] = 10
    edges = nx.draw_networkx_edges(G, pos, ax=ax[i], connectionstyle=f'arc3, rad = {arc_rad}', edge_cmap=cm.Blues, width=width,
                edge_color=edge_color, arrows=True, arrowsize=10)
    
    ax[i].set_axis_off()
    ax[i].set_title(f"t = {i}", fontsize=50)

plt.tight_layout()

In [ ]:
plot_trajectories(trajectory_P,x_max=N)

In [ ]:

trajectory_Z = random_walk_reflecting(p,q,num_steps=T, x_min=0, x_max=N, boundary_type='stick')

plot_trajectories(trajectory_Z,x_max=N)

In [ ]:
Nmin = 5000
final_positions_P = trajectory_P[Nmin:]
final_positions_Z = trajectory_Z[Nmin:]

In [ ]:

fig, ax = plt.subplots(1,2,figsize=(10, 4), sharex=True, sharey=True)
ax[0].hist(final_positions_Z, bins = np.arange(0.5,N+1.5,1), density=True, color='b', alpha=0.7, edgecolor='black',label='Empirical')  # Probability distribution
ax[0].scatter(xs,ys0,label='Theory',s=100, c='r')
ax[0].set_title(f'Long run probability using P')
ax[0].set_xlabel('State')
ax[0].set_ylabel('Probability')
ax[0].legend(loc='best')
ax[0].grid(True)

ax[1].hist(final_positions_P, bins = np.arange(0.5,N+1.5,1), density=True, color='b', alpha=0.7, edgecolor='black',label='Empirical')  # Probability distribution
ax[1].scatter(xs,ys0,label='Theory',s=100, c='r')
ax[1].set_title(f'Long run probability using Z')
ax[1].set_xlabel('State')
ax[1].set_ylabel('Probability')
ax[1].legend(loc='best')
ax[1].grid(True)

# plt.show()

### Number of Visits
We can check what is the expected value of the number of visits to node $j$, given we start at $i$:

$\mathbb{E} [N_{i}(j)] = \frac{r_{ij}}{1-r_{jj}} \, = \sum_{n=1}^{\infty}(P^{n})_{ij}$

For this, we can compute the RHS using the power of a matrix

In [ ]:
from numpy.linalg import matrix_power

In [ ]:
P

First, let's take a look at $P^n$, tuning $n$ from small to large values

In [ ]:
max_N = 1001
P_to_the_N = matrix_power(P, max_N)
P_to_the_N[:2]

In [ ]:
fs = 10
plt.figure(figsize=(6,3))
fig, ax = plt.subplots(1,2,figsize=(10,5))

# Original transition matrix
ax[0].imshow(P, vmax=1,vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(P):
    ax[0].text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
    ax[0].text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)

ax[0].axis('off')
ax[0].set_title('P')
# n-step transition matrix
ax[1].imshow(P_to_the_N, vmax=1,vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(P_to_the_N):
    ax[1].text(i,j,f"{label:.3f}",ha='center',va='center', c='black', fontsize = fs)
    ax[1].text(i,j,f"{label:.3f}",ha='center',va='center', c='black', fontsize = fs)
    
ax[1].axis('off')
ax[1].set_title(r'$P^n$' +f", n = {max_N}" )
# pc.colorbar()

### Q3: play with `max_N`, what do you see?

Now we are ready to take the **sum** of this over various $n$

In [ ]:
max_N = 1000
sumP_to_the_N = sum([matrix_power(P, n) for n in range(1,max_N+1)])

sumP_to_the_N[:3]

In [ ]:
fs = 10
plt.figure(figsize=(6,3))
fig, ax = plt.subplots(1,2,figsize=(10,5))

# Original transition matrix
ax[0].imshow(P, vmax=1,vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(P):
    ax[0].text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
    ax[0].text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)

ax[0].axis('off')
ax[0].set_title('P')
# n-step transition matrix
ax[1].imshow(sumP_to_the_N, cmap='Blues')

for (j,i),label in np.ndenumerate(sumP_to_the_N):
    ax[1].text(i,j,f"{label:.1f}",ha='center',va='center', c='black', fontsize = fs)
    ax[1].text(i,j,f"{label:.1f}",ha='center',va='center', c='black', fontsize = fs)
    
ax[1].axis('off')
ax[1].set_title(r'$\sum_{n=1}^{max_N} (P^n)$' +f", max_N = {max_N}" )
# pc.colorbar()

### Find steady state distribution

**Method 1**: solving an eigenvector equation.

**Note**: you can sometime do this also _analytically_, by using a system of equations. This can be helpful in simple cases, e.g. when there are 2-3 states.

In [ ]:
# Convert the transition matrix to a NumPy array
transition_matrix = np.array(P)

# Calculate the steady-state distribution
eigenvalues, eigenvectors = np.linalg.eig(P.T)
steady_state = eigenvectors[:, np.isclose(eigenvalues, 1)]

# Normalize the steady-state distribution
steady_state = steady_state / steady_state.sum()

print(steady_state.real.flatten())

In [ ]:
ys = steady_state.real.flatten()
fs = 20
plt.figure(figsize=(6,3))

plt.scatter(xs,ys)
plt.text( N-2,(max(ys)- 0.1*(max(ys)-min(ys))),f"{r'$\rho=$'}{p/q:.4f}")
plt.xlabel('State i',fontsize=fs)
plt.ylabel(r'$\pi_i$',fontsize=fs)
plt.show()

**Method 2**: using the power of $P$ (and its relations with the expected number of visited sites)


$ \lim_{n \rightarrow \infty} \frac{\mathbb{E} [N_{n,i}(j)]}{n}= \pi_{j}$

In [ ]:
max_N = 10000
sumP_to_the_N = sum([matrix_power(P, n)/max_N for n in range(1,max_N)])

sumP_to_the_N[:3]

In [ ]:
sumP_to_the_N[i]

In [ ]:
i = 0
assert np.allclose(sumP_to_the_N[i],steady_state.real.flatten(),rtol=1e-1)

In [ ]:
ys2 = sumP_to_the_N[0].real.flatten()
fs = 20
plt.figure(figsize=(6,3))

plt.scatter(xs,ys0,marker='D',label='Theory')
plt.scatter(xs,ys,label='Method 1: eigenvalues')
plt.scatter(xs,ys2,marker='x',label='Method 2: sumP_to_the_N',alpha=0.5)
plt.text( N-2,(max(ys)- 0.1*(max(ys)-min(ys))),f"{r'$\rho=$'}{p/q:.4f}")
plt.xlabel('State i',fontsize=fs)
plt.ylabel(r'$\pi_i$',fontsize=fs)
plt.legend()
plt.show()

# Gambler's ruin
This can be seen as a random walk as above with absorbing boundary conditions at 0 and $m$.  
$m$ is the sum we fix to end the game (if we win).  
We assume that $r=0$, i.e. there is no resting step.  


In [ ]:
m = 10
p = 0.5
q = 0.5
rho = p / q
print(f"rho={rho:.3f}")

### What is the probability of ruin?
There are two possibilities, depending on $\rho$:
1. $r_{a} = \frac{\rho^{a}-\rho^{m}}{	1-\rho^{m}}\,,\quad (\rho\neq1)$
2. $r_{a} = 1-\frac{a}{m}\,,\quad(\rho=1)$

Let's check how this behaves

In [ ]:
def calculate_ra(a: int, m: int, rho, EPS=1e-8):
    assert 0<a<m, f"a={a}"
    if np.isclose(rho,1):
        return 1 - a/m
    num = np.power(rho,a) - np.power(rho,m)
    den = 1 - np.power(rho,m) + EPS
    return num / den

In [ ]:
m = 100 # value of the win
a = 10
xs = np.linspace(0.001,2,100)
ys0 = [calculate_ra(a,m, rho=rho) for rho in xs]
assert np.allclose(np.sum(ys),1)

In [ ]:
fs = 20
plt.figure(figsize=(6,3))

plt.scatter(xs,ys0)
plt.text( 0.1,0.9,f"a={a:.0f}, m={m}")
plt.axvline(x=1, color='b', linestyle='--', lw=1)
plt.axhline(y=1, color='r', linestyle='--', lw=1)

plt.xlabel(r'$\rho$',fontsize=fs)
plt.ylabel(r'$r_a$',fontsize=fs)
plt.show()

What happens if we send $m\rightarrow \infty$?

In [ ]:
m = 1000 # value of the win
a = 10
xs = np.linspace(0.001,2,100)
ys0 = [calculate_ra(a,m, rho=rho) for rho in xs]
assert np.allclose(np.sum(ys),1)

fs = 20
plt.figure(figsize=(6,3))

plt.scatter(xs,ys0)
plt.text( 0.1,0.9,f"a={a:.0f}, m={m}")
plt.axvline(x=1, color='b', linestyle='--', lw=1)
plt.axhline(y=1, color='r', linestyle='--', lw=1)

plt.xlabel(r'$\rho$',fontsize=fs)
plt.ylabel(r'$r_a$',fontsize=fs)
plt.show()

### Compare with simulations

Let's test the theory by running some simulations and comparing with it

In [ ]:
def get_P_gambler(p: float, q: float, m: int = 10):

    assert np.isclose(p+q,1)
    
    P = np.zeros((m+1,m+1))

    # All other entries
    for i in range(1,m):
        P[i,i-1] = q
        P[i,i+1] = p

    # Boundaries
    P[0,0] = 1
    P[0,1] = 0
    P[m,m-1] = 0
    P[m,m] = 1

    return P

In [ ]:
m = 10
p = 0.5
q = 0.5

In [ ]:
P_gambler = get_P_gambler(p,q,m)
assert np.all(np.isclose(np.sum(P_gambler,axis=1),1)),np.sum(P_gambler,axis=1)
print(P_gambler.shape)

In [ ]:
fs = 10
plt.figure(figsize=(8,5))
plt.imshow(P_gambler[:10,:10], vmax=1,vmin=0, cmap='Blues')

if m <=10:
    for (j,i),label in np.ndenumerate(P_gambler):
        plt.text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
        plt.text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
    plt.axis('off')
    plt.colorbar()

In [ ]:
initial_state = 9
T = 1000
trajectory_P_gambler = get_trajectory_with_P(initial_state,P_gambler, T, seed=11)
trajectory_P_gambler.shape

In [ ]:
plot_trajectories(trajectory_P_gambler,x_max=m)

Let's now run for many gamblers

In [ ]:

initial_state = 5
N_gamblers = 1000
T = 1000
trajectory_P_gamblers = np.array([get_trajectory_with_P(initial_state,P_gambler, T, seed=seed) for seed in np.arange(N_gamblers)])
trajectory_P_gamblers.shape

In [ ]:
print('Example ends of the trajectory:') 
trajectory_P_gamblers[:10,-1]

In [ ]:
np.unique(trajectory_P_gamblers[:,-1], return_counts=True)

In [ ]:
n_wins = np.sum(trajectory_P_gamblers[:,-1] == m) / N_gamblers
n_ruins = np.sum(trajectory_P_gamblers[:,-1] == 0) / N_gamblers
# assert n_wins == 1-n_ruins, f"n_wins={n_wins}, n_ruins={n_ruins}"
print(f"n_wins = {n_wins}, n_ruins = {n_ruins}")

ra = calculate_ra(initial_state,m, rho=p/q) 
print(f"theory: {ra:.2f}")

### Duration of the game
The theory says that the expected value $d_a = \text{E}[T]$ of the duration of a game, where:

$T=\min \{n>0 | X_{n}=0 \lor X_{n}=m\}$ 

is: 

$
d_{a}=\left\{
\begin{array}{ll}
\frac{1}{q-p}(a-m\,\frac{1-\rho^{a}}{1-\rho^{m}}) & (\rho \neq 1)\\
a\,(m-a) & (\rho=1)
\end{array}
\right.
$

Let's use the simulations above to check this

In [ ]:
def get_expected_duration(p: float,q: float, m: int, a: int = 0):

    assert 0<p<1
    assert 0<q<1
    assert 0<a<m
    
    rho = p/q
    if np.isclose(rho,1):
        return a * (m-a)
    d = a - m * (1 - np.power(rho,a)) / (1 - np.power(rho,m))
    d = d / (q-p)
    return d

In [ ]:
assert N_gamblers == len(np.where(trajectory_P_gamblers[:,-1] == m)[0]) + len(np.where(trajectory_P_gamblers[:,-1] == 0)[0])

In [ ]:
duration = np.zeros(N_gamblers)
for i in range(N_gamblers):
    cond_win = np.where(trajectory_P_gamblers[i] == m)[0]
    if len(cond_win) == 0:
        cond_ruin = np.where(trajectory_P_gamblers[i] == 0)[0]
        duration[i] = min(cond_ruin)
    else:
        duration[i] = min(cond_win)
    

In [ ]:
expected_d = get_expected_duration(p,q,m,initial_state)
empirical_mean_duration = np.mean(duration)

fig, ax = plt.subplots(1,1,figsize=(6, 4))
ax.hist(duration, bins = 20, density=True, color='b', alpha=0.7, edgecolor='black')  # Probability distribution
plt.axvline(x=expected_d, color='r', linestyle='--', lw=1, label='Theory')
plt.axvline(x=empirical_mean_duration, color='b', linestyle='--', lw=1,label='Empirical')

ax.set_title(f'p={p:.2f}, q={q:.2f}\na={initial_state}, m={m}')
ax.set_xlabel('Duration')
ax.set_ylabel('Probability')
ax.legend(loc='best')
ax.grid(True)

# plt.show()